In [ ]:
import cv2 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = [10, 15]

### Loading images

In [ ]:
trump_ten = cv2.imread("./inputs/P5/donald_plays_tennis.png")
trump_golf = cv2.imread("./inputs/P5/donald_plays_golf.png")

COST = np.array([])
IMG_WITH_SEAM = np.array([])
GRAD = np.array([])

### Asked Functions

In [ ]:
def find_energy_map(img):
    # input images are BGR
    gray_scale = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return sum(map(abs,np.gradient(gray_scale)))


In [ ]:
def get_cost_matrix(energy_map,i,j,cost=None,is_proc=None):
    #check to possible indexes
    if i <0 or j<0 or i>= energy_map.shape[0] or j >= energy_map.shape[1]:
        return cost
    
    # if start of calculation
    if is_proc is None :
        is_proc = np.zeros_like(energy_map)
        cost = np.zeros_like(energy_map)
        # check to fill entire
        for _ in range(energy_map.shape[1]):
            cost = get_cost_matrix(energy_map,i,_,cost,is_proc)
        return cost
    
    # check if calculated before       
    if is_proc[i,j] == 1 or i-1 < 0:
        return cost
    
    # collect top-3 cell
    values = []
    if  j-1 >= 0 :
        values.append(get_cost_matrix(energy_map,i-1,j-1,cost,is_proc)[i-1,j-1])
    if  j+1 < energy_map.shape[1] :
        values.append(get_cost_matrix(energy_map,i-1,j+1,cost,is_proc)[i-1,j+1])
    
    values.append(get_cost_matrix(energy_map,i-1,j,cost,is_proc)[i-1,j])
     
    # update matrix   
    cost[i,j] = energy_map[i,j] + min(values)
    is_proc[i,j] = 1
    return cost


In [ ]:
def find_seams_list(energy_map,seam_dir, seam_num,img):
    img = img.copy()
    
    # check direction to calculation
    if seam_dir != "vertical":
        energy_map = cv2.rotate(energy_map, cv2.ROTATE_90_CLOCKWISE) 
        img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE) 
        
    #store visual reports global
    global GRAD
    global COST 
    global IMG_WITH_SEAM 
    GRAD = energy_map
        
    #calculate Tottal cost matrix(M)
    cost = get_cost_matrix(energy_map.copy(),energy_map.shape[0]-1,0)
    
    _cost = cost.copy()
    if seam_dir != "vertical":     
        _cost = cv2.rotate(_cost, cv2.ROTATE_90_COUNTERCLOCKWISE)
    COST = _cost
    
    # result report
    result = np.zeros((seam_num,cost.shape[0]))
    
    # a mask to control which cells(pixles) is deleted before
    mask = np.zeros_like(cost)
    
    # sort last row/col to find minimal energy
    sorted_cost = cost [cost.shape[0]-1,:].argsort()
    
    # select seams loop
    for index in range(seam_num):
        # select next postion to strat seam
        j = sorted_cost[index]
        
        # update first postion in reports
        img [cost.shape[0]-1,j,:] = [0,0,255]
        GRAD[cost.shape[0]-1,j] = 255
        mask [cost.shape[0]-1,j] = index + 1
        
        # a loop to select path
        for row in range(cost.shape[0]-2,-1 ,-1):
            # if top-right/left cell available
            left,right = False,False
            
            # select top 3 pixles
            tops = []
            
            # if exact top available and not selected before
            if mask [row,j] == 0:
                tops.append(cost[row,j])
            
            # select first top-left pixle that not selected before 
            j_left = j - 1
            while j_left != -1 and mask [row,j_left] != 0 :
                j_left-=1
                if j_left == -1 :
                    break 
             
            # select first top-right pixle that not selected before 
            j_right = j + 1
            while j_right != cost.shape[1] and mask [row,j_right] != 0 :
                j_right+=1
                if j_right == cost.shape[1] :
                    j_right = -1
                    break 
                
            # get value if available  
            if j_left != -1 :            
                tops.append(cost[row,j_left])
                left= True
            
            # get value if available      
            if j_right != -1 and j_right != cost.shape[1]:            
                tops.append(cost[row,j_right])
                right= True
            
            # find which top is minimum    
            if left and min(tops) == cost[row,j_left]:
                j = j_left
            elif right and min(tops) == cost[row,j_right]:
                j = j_right 
            
            #update report
            img [row,j,:] = [0,0,255]
            GRAD[row,j] = 255
            mask [row,j] = index + 1
        
        # convert result ro asked format    
        result[index,:] = np.array(list((zip(np.where(mask==index + 1)[0],
                                             np.where(mask==index + 1)[1]))))[:,1]   
    if seam_dir != "vertical":     
        img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
        GRAD = cv2.rotate(GRAD, cv2.ROTATE_90_COUNTERCLOCKWISE)
        
    IMG_WITH_SEAM = img
    return result
        
 

In [ ]:
def remove_seams(img, seams_list, seam_dir=""):
    img = img.copy()
    if seam_dir != "vertical":
        img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE) 
    
    # generate a mask based on given seams list
    mask = np.ones(shape=img.shape,dtype=np.bool8)
    
    # empty images that will be filled 
    out = np.zeros((img.shape[0],img.shape[1]-seams_list.shape[0],3),dtype=np.uint8)

    # complte the mask
    for i in range(seams_list.shape[0]):
        for j in range(seams_list.shape[1]):
            mask[j,int(seams_list[i][j]),:] = False
    
    # select image area that masked
    for i in range(seams_list.shape[1]):
        out[i,:,:] = img[i,:,:][mask[i,:,0]]
        
    if seam_dir != "vertical": 
        out = cv2.rotate(out, cv2.ROTATE_90_COUNTERCLOCKWISE)
        
    return out 
    

In [ ]:
def add_seams(img, seams_list, seam_dir=""):
    img = img.copy()
    if seam_dir != "vertical":
        img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE) 
    
    # generate a mask based on given seams list
    mask = np.zeros(shape=img.shape,dtype=np.bool8)
    
    # empty images that will be filled 
    out = np.zeros((img.shape[0],img.shape[1]+seams_list.shape[0],3),dtype=np.uint8)

    # complte the mask
    for i in range(seams_list.shape[0]):
        for j in range(seams_list.shape[1]):
            mask[j,int(seams_list[i][j]),:] = True
    
    # select image area that masked
    for i in range(mask.shape[0]):
        j = -1
        for k in range(mask.shape[1]): 
            j += 1            
            out[i,j,:] = img[i,k,:]
            
            if mask [i,k,0]:
                out[i,j+1,:] = img[i,k,:]
                j += 1


        
    if seam_dir != "vertical": 
        out = cv2.rotate(out, cv2.ROTATE_90_COUNTERCLOCKWISE)
        
    return out 
    


In [ ]:
def Handler(img,dir_,percentage,show=True,name="",energy_map=None,add=False):
    if energy_map is None:
        energy_map = find_energy_map(img)
    energy = energy_map.copy()
    if dir_ == "vertical":
        res = find_seams_list(energy,dir_,int(energy_map.shape[1]*percentage)+1,img)
    else:
        res = find_seams_list(energy,dir_,int(energy_map.shape[0]*percentage)+1,img)
        
    if not add:
        out = remove_seams(img,res,dir_)
        out_2 = remove_seams(cv2.cvtColor(energy_map.astype("uint8").copy(),cv2.COLOR_GRAY2BGR),res,dir_)
    else:
        out = add_seams(img,res,dir_)
        out_2 = add_seams(cv2.cvtColor(energy_map.astype("uint8").copy(),cv2.COLOR_GRAY2BGR),res,dir_)
    if not show:
        return out
    plt.clf()   
    plt.rcParams['figure.figsize'] = [10, 15]
    plt.subplot(4,2,1)
    plt.imshow(img[:,:,::-1])
    plt.axis("off")
    plt.title("Orginal Iamge")
    plt.subplot(4,2,2)
    plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2GRAY),cmap="gray")
    plt.axis("off")
    plt.title("Gray-scale Iamge")
    plt.subplot(4,2,3)
    plt.imshow(energy_map,cmap="gray")
    plt.axis("off")
    plt.title("Energy Map")
    plt.subplot(4,2,4)
    plt.imshow(GRAD,cmap="gray")
    plt.axis("off")
    plt.title("Energy Map with selected seams")
    
    plt.subplot(4,2,5)
    plt.imshow(IMG_WITH_SEAM[:,:,::-1])
    plt.axis("off")
    plt.title("Orginal Image with selected seams")
    plt.subplot(4,2,6)
    plt.imshow(COST/np.max(COST),cmap="cool")
    plt.axis("off")
    plt.title("Result of M-Matrix which calculated by DP")


    plt.subplot(4,2,7)
    plt.imshow(out_2)
    plt.axis("off")
    plt.title("Resized energy map".format(percentage,dir_))
    
    plt.subplot(4,2,8)
    plt.imshow(out[:,:,::-1])
    plt.axis("off")
    plt.title("Resized image".format(percentage,dir_))
    

    plt.tight_layout()
    plt.savefig("./outputs/P05/"+name+".png")
    plt.show()
    return out
    


### A

In [ ]:
ratios = [0.1,0.25,0.5]
for r in ratios:
    test = trump_ten.copy()
    out = Handler(test,"vertical",r,True,"Ten-single-step-{0}".format(r))


### A - one-by-one

In [ ]:
ratios = [0.1,0.25,0.5]
results_ten = []
test = trump_ten.copy()
for r in ratios:
    print("Proc:",r)
    for i in range(5000):
        test = Handler(test,"vertical",0.00000000001,False)
        if test.shape[1]/trump_ten.shape[1] <= 1-r:
            break
    results_ten.append(test)

### B

In [ ]:
ratios = [0.1,0.25,0.5]
for r in ratios:
    test = trump_golf.copy()
    out = Handler(test,"horizontal",r,True,"Golf-single-step-{0}".format(r))


### B - one-by-one

In [ ]:
ratios = [0.1,0.25,0.5]
results_golf = []
test = trump_golf.copy()
for r in ratios:
    print("Proc:",r)
    for i in range(5000):
        test = Handler(test,"horizontal",0.00000000001,False)
        if test.shape[0]/trump_golf.shape[0] <= 1- r:
            break
    results_golf.append(test)

In [ ]:
#cv2.imwrite("./outputs/P05/Golf-multi-step.png",cv2.vconcat(results_golf))
cv2.imwrite("./outputs/P05/Ten-multi-step.png",cv2.hconcat(results_ten))

### C

In [ ]:
img = cv2.imread("./inputs/c-2.jpg")
out = Handler(img,"vertical",0.2,True,"C")

### D

In [ ]:
img = cv2.imread("./inputs/d.jpg")
out = Handler(img,"vertical",0.1,True,"D")

### E

In [ ]:
trump_ten = cv2.imread("./inputs/P5/donald_plays_tennis.png")
trump_golf = cv2.imread("./inputs/P5/donald_plays_golf.png")
putin =  cv2.imread("./inputs/P5/trump_putin.png")
queen = cv2.imread("./inputs/P5/trump_queen.png")

ratios = [0.1,0.25,0.5] 
for r in ratios:
    test = trump_ten.copy()
    out = Handler(test,"vertical",r,True,"E-increas-{0}-{1}".format("Trump Ten",r),add=True)
    
for r in ratios:
    test = putin.copy()
    out = Handler(test,"vertical",r,True,"E-increas-{0}-{1}".format("Putin",r),add=True)
    
for r in ratios:
    test = queen.copy()
    out = Handler(test,"vertical",r,True,"E-increas-{0}-{1}".format("queen",r),add=True)
    
for r in ratios:
    test = trump_golf.copy()
    out = Handler(test,"horizontal",r,True,"E-increas-{0}-{1}".format("Trump Golf",r),add=True)


### F

In [ ]:
putin = cv2.imread("./inputs/P5/trump_putin.png")
putin_mask = cv2.imread("./inputs/P5/trump_putin_mask.png")

queen = cv2.imread("./inputs/P5/trump_queen.png")
queen_mask = cv2.imread("./inputs/P5/trump_queen_mask.png")


GRAD = find_energy_map(queen)+(-5)*(queen_mask[:,:,0])
out = Handler(queen,"vertical",0.14,True,"F-queen",GRAD)
while np.where(GRAD<0)[0].shape[0] <= 1 :
    out = Handler(out,"vertical",0.00000001,False,"F-queen",GRAD)
out = Handler(out,"vertical",0.00000001,False,"F-queen",GRAD)
print("Queen Masked")

GRAD = find_energy_map(putin[1:,:,:])+(-5)*(putin_mask[:,:,0])
out = Handler(putin[1:,:,:],"vertical",0.2825,True,"F-putin",GRAD)
while np.where(GRAD<0)[0].shape[0] <= 1 :
    out = Handler(out,"vertical",0.00000001,False,"F-putin",GRAD)
out = Handler(out,"vertical",0.00000001,False,"F-putin",GRAD)
print("Putin Masked")
